**Import các thư viện cần thiết**

In [1]:
import math
import numpy as np
from scipy.spatial import distance
from sklearn.feature_extraction.text import TfidfVectorizer

**Tập dữ liệu văn bản/tài liệu mẫu và truy vấn**

In [2]:
# Tập văn bản/tài liệu (D)
docs = [
    'data mining is awesome. data mining helps to find frequent itemsets in database.',
    'information retrieval is cool. information retrieval helps to search data quickly.',
    'natural language processing is interesting. it helps computer to better understand text.'
];

# Truy vấn (q)
query = 'information retrieval'

# Để cho tiện lợi trong việc xử lý chúng ta sẽ gán câu truy vấn là 1 tài liệu cuối cùng
docs.append(query)

doc_len = len(docs)


**Thực hiện biến đổi các tài liệu/văn bản về dạng các TF-IDF vectors bằng cách dùng thư viện Scikit-Learn - TfidfVectorizer**

In [3]:
# Khởi tạo đối tượng TfidfVectorizer
vectorizer = TfidfVectorizer()

# Tiến hành chuyển đổi các tài liệu/văn bản và truy vấn về dạng các vector TF-IDF
# [tfidf_matrix] là một ma trận ở dạng thưa (sparse) - chỉ lưu các vị trí có giá trị khác 0  - chứa trọng số TF-IDF của các tài liệu/văn bản
tfidf_matrix = vectorizer.fit_transform(docs)

# Chúng ta tiến hành chuyển ma trận tfidf_matrix về dạng đầy đủ
tfidf_matrix = tfidf_matrix.todense()

# Lấy danh sách tập từ vựng
vocab = vectorizer.get_feature_names_out()
vocab_size = len(vocab)
print('Kích thước tập từ vựng: [{}]'.format(vocab_size))
print('Tập từ vựng (V):')
print(vocab)

# Chuyển đổi  ma trận (numpy) về dạng list
tfidf_matrix_list = tfidf_matrix.tolist()

# TFIDF encode của truy vấn (q) là tài liệu cuối cùng
# Kết quả tính TF-IDF của thư viện Scikit-Learn sẽ hơi khác với cách tính truyền thống
# vì IDF của Scikit-Learn sẽ là: idf(t) = loge [ (1+n) / ( 1 + df ) ] + 1
# sau đó toàn bộ ma trận TF-IDF sẽ được bình thường hoá lại với (norm - L2)
# tuy nhiên kết quả cuối cùng cũng sẽ không thay đổi
query_tfidf_encoded_vector = tfidf_matrix_list[doc_len-1]
print(query_tfidf_encoded_vector)

# Xóa query đã được mã hóa thành dạng tfidf vector ra khỏi tfidf_matrix_list
del tfidf_matrix_list[doc_len-1]

Kích thước tập từ vựng: [25]
Tập từ vựng (V):
['awesome' 'better' 'computer' 'cool' 'data' 'database' 'find' 'frequent'
 'helps' 'in' 'information' 'interesting' 'is' 'it' 'itemsets' 'language'
 'mining' 'natural' 'processing' 'quickly' 'retrieval' 'search' 'text'
 'to' 'understand']
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, 0.0, 0.0, 0.0]


**Tìm danh sách các tài liệu liên quan đến truy vấn (Tích vô hướng, khoảng cách Euclid và Tương đồng cosine)**

In [4]:
# Duyệt qua danh sách các tài liệu/văn bản (đã mã hóa ở dạng  vectors)
for doc_idx, doc_tfidf_encoded_vector in enumerate(tfidf_matrix_list):

  # Tính tích vô hướng giữa hai vectors tài liệu và truy vấn
  dot_product_sim = np.dot(query_tfidf_encoded_vector, doc_tfidf_encoded_vector)

  # Tính khoảng cách Euclid giữa hai vectors tài liệu và truy vấn
  ed = distance.euclidean(query_tfidf_encoded_vector, doc_tfidf_encoded_vector)
  
  # Tính tương đồ cosine giữa hai vectors tài liệu và truy vấn
  cs = 1 - distance.cosine(query_tfidf_encoded_vector, doc_tfidf_encoded_vector)

  print('Tài liệu: [{}], tương đồng (dot product): [{:.6f}]'.format(doc_idx, dot_product_sim))
  print('Tài liệu: [{}], tương đồng (khoảng cách Euclid): [{:.6f}]'.format(doc_idx, ed))
  print('Tài liệu: [{}], tương đồng (Tương đồng cosine): [{:.6f}]'.format(doc_idx, cs))
  print('---')

Tài liệu: [0], tương đồng (dot product): [0.000000]
Tài liệu: [0], tương đồng (khoảng cách Euclid): [1.414214]
Tài liệu: [0], tương đồng (Tương đồng cosine): [0.000000]
---
Tài liệu: [1], tương đồng (dot product): [0.711735]
Tài liệu: [1], tương đồng (khoảng cách Euclid): [0.759296]
Tài liệu: [1], tương đồng (Tương đồng cosine): [0.711735]
---
Tài liệu: [2], tương đồng (dot product): [0.000000]
Tài liệu: [2], tương đồng (khoảng cách Euclid): [1.414214]
Tài liệu: [2], tương đồng (Tương đồng cosine): [0.000000]
---
